In [7]:
!pip install ray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 MB 9.3 MB/s eta 0:00:00


In [18]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:           6
    Model:                79
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4399.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht sysc
                          all nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xt
                          opology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq
                           ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt
                           aes xsave avx f16c rdrand hypervisor 

In [8]:
%%python
import ray
ray.init()

@ray.remote
def hello_world():
    return "Ray is working!"

result = ray.get(hello_world.remote())
print(result)

Ray is working!


2025-08-30 00:04:42,882	INFO worker.py:1951 -- Started a local Ray instance.


In [11]:
import ray

def add(a,b):
  return a+b

@ray.remote
def remote_add(a,b):
  return a+b

add, remote_add

(<function __main__.add(a, b)>,
 <ray.remote_function.RemoteFunction at 0x7b92b3ccbe60>)

Invoke function remotely.

In [17]:
object_ref = remote_add.remote(1,2)
ray.get(object_ref)

3

In [40]:
import sys
import numpy as np
import os
import time


In [23]:
@ray.remote
def sqrt_add(a,b):
  return np.sqrt(a) + b

ray.get([sqrt_add.remote(1,2), sqrt_add.remote(4,5)])


[np.float64(3.0), np.float64(7.0)]

In [29]:
jobs = []
for i in range(20):
  jobs.append(sqrt_add.remote(float(i**i), float(i**3)))

ray.get(jobs)

[np.float64(1.0),
 np.float64(2.0),
 np.float64(10.0),
 np.float64(32.19615242270663),
 np.float64(80.0),
 np.float64(180.90169943749476),
 np.float64(432.0),
 np.float64(1250.4926996951544),
 np.float64(4608.0),
 np.float64(20412.0),
 np.float64(101000.0),
 np.float64(535476.7391115275),
 np.float64(2987712.0),
 np.float64(17405504.346371062),
 np.float64(105416248.0),
 np.float64(661738888.9184079),
 np.float64(4294971392.0),
 np.float64(28761789660.93136),
 np.float64(198359296200.0),
 np.float64(1406563071801.4553)]

In [32]:
large_matrix = np.random.rand(1024, 1024, 1024//8) # approx. 1 GB
size_in_bytes = sys.getsizeof(large_matrix)

print(f"large_matrix has: {size_in_bytes/1024/1024/1024:.2f} GB")

large_matrix has: 1.00 GB


In [33]:
obj_ref = ray.put(large_matrix)

In [34]:
large_mat_from_object_store = ray.get(obj_ref)


In [35]:
np.array_equal(large_mat_from_object_store, large_matrix)


True

Chaining tasks

In [37]:
@ray.remote
def sq(a):
  return a**2

@ray.remote
def cube(a):
  return a**3

r = sq.remote(2)
r = cube.remote(r)
ray.get(r)

64

In [38]:
ray.available_resources()


{'CPU': 2.0,
 'object_store_memory': 2924238582.0,
 'node:172.28.0.12': 1.0,
 'node:__internal_head__': 1.0,
 'memory': 9328621568.0}

In [39]:
@ray.remote
def expensive_square(x):
    time.sleep(np.random.randint(1, 10))
    return x**2

In [41]:
expensive_compute = []

for i in range(15):
    expensive_compute.append(expensive_square.remote(i))

expensive_compute

[ObjectRef(852304542258e48cffffffffffffffffffffffff0100000001000000),
 ObjectRef(96eec688321e7882ffffffffffffffffffffffff0100000001000000),
 ObjectRef(e5b2fae6e42f6bbaffffffffffffffffffffffff0100000001000000),
 ObjectRef(74701ed4b46295a6ffffffffffffffffffffffff0100000001000000),
 ObjectRef(b53defeececf8bc6ffffffffffffffffffffffff0100000001000000),
 ObjectRef(641105d6352e1cecffffffffffffffffffffffff0100000001000000),
 ObjectRef(f173ee62d1d97be7ffffffffffffffffffffffff0100000001000000),
 ObjectRef(692f535a66884eb7ffffffffffffffffffffffff0100000001000000),
 ObjectRef(e36c19e9e03caebeffffffffffffffffffffffff0100000001000000),
 ObjectRef(2ca6f68d2b906770ffffffffffffffffffffffff0100000001000000),
 ObjectRef(ff79a3c11645664affffffffffffffffffffffff0100000001000000),
 ObjectRef(ea19b1b85fbbce81ffffffffffffffffffffffff0100000001000000),
 ObjectRef(4abf1de3f84b576dffffffffffffffffffffffff0100000001000000),
 ObjectRef(e593db9b22e29786ffffffffffffffffffffffff0100000001000000),
 ObjectRef(2587074a1

In [42]:
ready_refs, not_ready_refs = ray.wait(expensive_compute) # wait for next object ref that is ready

# process new item as soon as it becomes available
while not_ready_refs:
    print(f"{ready_refs[0]} is ready; result: {ray.get(ready_refs[0])}")
    print(f"{len(not_ready_refs)} items not ready... \n")

    ready_refs, not_ready_refs = ray.wait(not_ready_refs) # wait for next item

    assert len(ready_refs) == 1, f"len(ready_refs) should be 1, got {len(ready_refs)} instead"

print(f"I'm the last item: {ready_refs[0]}; result: {ray.get(ready_refs[0])}")

ObjectRef(852304542258e48cffffffffffffffffffffffff0100000001000000) is ready; result: 0
14 items not ready... 

ObjectRef(96eec688321e7882ffffffffffffffffffffffff0100000001000000) is ready; result: 1
13 items not ready... 

ObjectRef(e5b2fae6e42f6bbaffffffffffffffffffffffff0100000001000000) is ready; result: 4
12 items not ready... 

ObjectRef(74701ed4b46295a6ffffffffffffffffffffffff0100000001000000) is ready; result: 9
11 items not ready... 

ObjectRef(b53defeececf8bc6ffffffffffffffffffffffff0100000001000000) is ready; result: 16
10 items not ready... 

ObjectRef(641105d6352e1cecffffffffffffffffffffffff0100000001000000) is ready; result: 25
9 items not ready... 

ObjectRef(f173ee62d1d97be7ffffffffffffffffffffffff0100000001000000) is ready; result: 36
8 items not ready... 

ObjectRef(692f535a66884eb7ffffffffffffffffffffffff0100000001000000) is ready; result: 49
7 items not ready... 

ObjectRef(2ca6f68d2b906770ffffffffffffffffffffffff0100000001000000) is ready; result: 81
6 items not re

In [43]:
BATCH_SIZE = 3

ready_refs, not_ready_refs = ray.wait(expensive_compute, num_returns=BATCH_SIZE)  # wait for BATCH_SIZE object refs

# process new item as soon as it becomes available
while not_ready_refs:
    print(f"{ready_refs} are ready; results: {ray.get(ready_refs)}")
    print(f"{len(not_ready_refs)} items not ready... \n")
    ready_refs, not_ready_refs = ray.wait(not_ready_refs, num_returns=BATCH_SIZE)  # wait for BATCH_SIZE object refs

print(f"Last batch {ready_refs}; result: {ray.get(ready_refs)}")

[ObjectRef(852304542258e48cffffffffffffffffffffffff0100000001000000), ObjectRef(74701ed4b46295a6ffffffffffffffffffffffff0100000001000000), ObjectRef(e36c19e9e03caebeffffffffffffffffffffffff0100000001000000)] are ready; results: [0, 9, 64]
12 items not ready... 

[ObjectRef(e5b2fae6e42f6bbaffffffffffffffffffffffff0100000001000000), ObjectRef(f173ee62d1d97be7ffffffffffffffffffffffff0100000001000000), ObjectRef(692f535a66884eb7ffffffffffffffffffffffff0100000001000000)] are ready; results: [4, 36, 49]
9 items not ready... 

[ObjectRef(641105d6352e1cecffffffffffffffffffffffff0100000001000000), ObjectRef(e593db9b22e29786ffffffffffffffffffffffff0100000001000000), ObjectRef(2587074a14f1fac4ffffffffffffffffffffffff0100000001000000)] are ready; results: [25, 169, 196]
6 items not ready... 

[ObjectRef(2ca6f68d2b906770ffffffffffffffffffffffff0100000001000000), ObjectRef(ff79a3c11645664affffffffffffffffffffffff0100000001000000), ObjectRef(4abf1de3f84b576dffffffffffffffffffffffff0100000001000000)] 